from 'Predicting Stock Prices in Python' tutorial video [https://youtu.be/PuZY9q-aKLw]

## import dependencies

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pandas_datareader as web
import datetime as dt

from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM

## load data

In [2]:
company = 'GM'

start = dt.datetime(2010, 1,1)
end = dt.datetime(2021,1,1)

data = web.DataReader(company, 'yahoo', start, end)

## Prepare Data

In [3]:
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(data['Close'].values.reshape(-1,1))

prediction_days = 45

x_train = []
y_train = []

for x in range(prediction_days, len(scaled_data)):
    x_train.append(scaled_data[x-prediction_days:x, 0])
    y_train.append(scaled_data[x, 0])
    
x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

## Build the model

In [4]:
model = Sequential()

model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=1)) # Prediction of the next closing value

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x_train, y_train, epochs=25, batch_size=32)

Train on 2502 samples
Epoch 1/25
2502/2502 [==============================] - 22s 9ms/sample - loss: 0.0219
Epoch 2/25
2502/2502 [==============================] - 9s 4ms/sample - loss: 0.0071
Epoch 3/25
2502/2502 [==============================] - 9s 4ms/sample - loss: 0.0064
Epoch 4/25
2502/2502 [==============================] - 8s 3ms/sample - loss: 0.0058
Epoch 5/25
2502/2502 [==============================] - 9s 4ms/sample - loss: 0.0052
Epoch 6/25
2502/2502 [==============================] - 9s 3ms/sample - loss: 0.0046
Epoch 7/25
2502/2502 [==============================] - 9s 3ms/sample - loss: 0.0044
Epoch 8/25
2502/2502 [==============================] - 9s 4ms/sample - loss: 0.0043
Epoch 9/25
2502/2502 [==============================] - 10s 4ms/sample - loss: 0.0042
Epoch 10/25
2502/2502 [==============================] - 11s 4ms/sample - loss: 0.0037
Epoch 11/25
2502/2502 [==============================] - 11s 4ms/sample - loss: 0.0034
Epoch 12/25
2502/2502 [==============

## Load Test Data

In [5]:
'''Test The Model Accuracy on Existing Data'''

test_start = dt.datetime(2020, 1, 1)
test_end = dt.datetime.now()

test_data = web.DataReader(company, 'yahoo', test_start, test_end)
actual_prices = test_data['Close'].values

total_dataset = pd.concat((data['Close'], test_data['Close']), axis=0)

model_inputs = total_dataset[len(total_dataset) - len(test_data) - prediction_days:].values
model_inputs = model_inputs.reshape(-1, 1)
model_inputs = scaler.transform(model_inputs)

## Make Predictions on Test Data

In [6]:
x_test = []

for x in range(prediction_days, len(model_inputs)):
    x_test.append(model_inputs[x-prediction_days:x, 0])
    
x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

predicted_prices = model.predict(x_test)
predicted_prices = scaler.inverse_transform(predicted_prices)

## Plot the test predictions

In [7]:
plt.figure(figsize=(12,8))
plt.plot(actual_prices, color="black", label=f"Actual {company} price")
plt.plot(predicted_prices, color="green", label=f"Predicted {company} price")
plt.title(f"{company} Share Price")
plt.xlabel('Time')
plt.ylabel(f'{company} Share Price')
plt.legend()
plt.show()

## Predict Next Day

In [8]:
real_data = [model_inputs[len(model_inputs) - prediction_days:len(model_inputs)+1, 0]]
real_data = np.array(real_data)
real_data = np.reshape(real_data, (real_data.shape[0], real_data.shape[1], 1))

#print(scaler.inverse_transform(real_data[-1]))

prediction = model.predict(real_data)
prediction = scaler.inverse_transform(prediction)
print(f"Prediction: {prediction}")

Prediction: [[58.308834]]
